In [9]:
import pythoncom
import win32com.client as winAPI

In [10]:
STAND_BY = 0
RECEIVED = 1


class XASessionEvents:
    login_state = STAND_BY

    def OnLogin(self, code, msg):
        XASessionEvents.login_state = RECEIVED
        print(msg)

    def OnDisconnect(self, code, msg):
        pass


class XAQueryEvents:
    query_state = STAND_BY

    def OnReceiveData(self, code):
        XAQueryEvents.query_state = RECEIVED

    def OnReceiveMessage(self, error, nMessageCode, szMessage):
        print(szMessage)

In [11]:
import datetime
import time

SERVER_PORT = 20001
SHOW_CERTIFICATE_ERROR_DIALOG = False
REPEATED_DATA_QUERY = 1
TRANSACTION_REQUEST_EXCESS = -21
TODAY = datetime.datetime.now().strftime('%Y%m%d')

if __name__ == "__main__":
    id = "ssuyan"
    password = "tndus1!!"
    certificate_password = "qkrtndus1!!"
    xa_session = winAPI.DispatchWithEvents("XA_Session.XASession", XASessionEvents)

    # demo.ebestsec.co.kr => 모의투자 
    # hts.ebestsec.co.kr => 실투자
    xa_session.ConnectServer("hts.ebestsec.co.kr", SERVER_PORT)
    xa_session.Login(id, password, certificate_password, SERVER_PORT, SHOW_CERTIFICATE_ERROR_DIALOG)

    while XASessionEvents.login_state is STAND_BY:
        pythoncom.PumpWaitingMessages()
    XASessionEvents.login_state = STAND_BY

로그인 성공


In [12]:
TR = "t8430"
xa_query = winAPI.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEvents)
xa_query.ResFileName = "C:\\eBEST\\xingAPI\\Res\\" + TR + ".res"

xa_query.SetFieldData("t8430InBlock", "gubun", 0, 1)

while True:
    ret = xa_query.Request(False)
    """ Receiving error message, keep requesting until accepted """
    if ret is TRANSACTION_REQUEST_EXCESS:  # -34
        time.sleep(0.8)
    else:
        break
""" Wait window's event message """
while XAQueryEvents.query_state is STAND_BY:
    pythoncom.PumpWaitingMessages()
XAQueryEvents.query_state = STAND_BY

정상적으로 조회가 완료되었습니다.


In [13]:
while True:
    ret = xa_query.Request(False)
    """ Receiving error message, keep requesting until accepted """
    if ret is TRANSACTION_REQUEST_EXCESS:  # -34
        time.sleep(0.8)
    else:
        break
""" Wait window's event message """
while XAQueryEvents.query_state is STAND_BY:
    pythoncom.PumpWaitingMessages()
XAQueryEvents.query_state = STAND_BY

정상적으로 조회가 완료되었습니다.


In [14]:
kospi_codes = [(xa_query.GetFieldData('t8430OutBlock', 'shcode', idx),
                xa_query.GetFieldData('t8430OutBlock', 'hname', idx))
               for idx in range(xa_query.GetBlockCount('t8430OutBlock'))]
print(kospi_codes)
print(len(kospi_codes))

[('000020', '동화약품'), ('000040', 'KR모터스'), ('000050', '경방'), ('000070', '삼양홀딩스'), ('000075', '삼양홀딩스우'), ('000080', '하이트진로'), ('000087', '하이트진로2우B'), ('000100', '유한양행'), ('000105', '유한양행우'), ('000120', 'CJ대한통운'), ('000140', '하이트진로홀딩스'), ('000145', '하이트진로홀딩스우'), ('000150', '두산'), ('000155', '두산우'), ('000157', '두산2우B'), ('000180', '성창기업지주'), ('000210', 'DL'), ('000215', 'DL우'), ('000220', '유유제약'), ('000225', '유유제약1우'), ('000227', '유유제약2우B'), ('000230', '일동홀딩스'), ('000240', '한국앤컴퍼니'), ('000270', '기아'), ('000300', '대유플러스'), ('000320', '노루홀딩스'), ('000325', '노루홀딩스우'), ('000370', '한화손해보험'), ('000390', '삼화페인트'), ('000400', '롯데손해보험'), ('000430', '대원강업'), ('000480', '조선내화'), ('000490', '대동'), ('000500', '가온전선'), ('000520', '삼일제약'), ('000540', '흥국화재'), ('000545', '흥국화재우'), ('000547', '흥국화재2우B'), ('000590', 'CS홀딩스'), ('000640', '동아쏘시오홀딩스'), ('000650', '천일고속'), ('000660', 'SK하이닉스'), ('000670', '영풍'), ('000680', 'LS네트웍스'), ('000700', '유수홀딩스'), ('000720', '현대건설'), ('000725', '현대건설우'), ('000760', '이화산업'

In [15]:
import pandas as pd
import numpy as np

kospi_array = np.array(kospi_codes)
print(kospi_array)

kospi_list = pd.Series(kospi_array[:,1], index=kospi_array[:, 0])
print(kospi_list)

[['000020' '동화약품']
 ['000040' 'KR모터스']
 ['000050' '경방']
 ...
 ['760003' '키움 코스닥 150 ETN']
 ['760004' '키움 인버스 미국달러']
 ['760005' '키움 INDXX 미국테크?']]
000020              동화약품
000040             KR모터스
000050                경방
000070             삼양홀딩스
000075            삼양홀딩스우
               ...      
760001      키움 미국달러선물 ET
760002    키움 코스피 200 ETN
760003    키움 코스닥 150 ETN
760004       키움 인버스 미국달러
760005    키움 INDXX 미국테크?
Length: 2048, dtype: object


In [16]:
import win32com.client
import pythoncom
import time


class XQuery_t1101:
    def __init__(self):
        super().__init__()
        self.is_data_received = False

    def OnReceiveData(self, tr_code):
        """
        이베스트 서버에서 ReceiveData 이벤트 받으면 실행되는 event handler
        """
        self.is_data_received = True
        name = self.GetFieldData("t1101OutBlock", "hname", 0)
        price = self.GetFieldData("t1101OutBlock", "price", 0)
        volume = self.GetFieldData("t1101OutBlock", "volume", 0)
        print("종목;{0},현재가;{1}, 누적거래량;{2}".format(name, price, volume))
        print("TR code => {0}".format(tr_code))

    def request(self):
        """
        이베스트 서버에 일회성 TR data 요청함.
        """
        self.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1101.res"  # RES 파일 등록
        self.SetFieldData("t1101InBlock", "shcode", 0, "005930")  # 삼성전자.
        err_code = self.Request(False)  # data 요청하기 --  연속조회인경우만 True

        if err_code < 0:
            print("error... {0}".format(err_code)) # data 요청하기 --  연속조회인경우만 True

    @classmethod
    def get_instance(cls):
        xq_t1101 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", cls)
        return xq_t1101


class XReal_S3_:
    def __init__(self):
        super().__init__()
        self.count = 0

    def OnReceiveRealData(self, tr_code):  # event handler
        """
        이베스트 서버에서 ReceiveRealData 이벤트 받으면 실행되는 event handler
        """
        self.count += 1
        stockcode = self.GetFieldData("OutBlock", "shcode")
        price = self.GetFieldData("OutBlock", "price")
        chetime = self.GetFieldData("OutBlock", "chetime")
        print(self.count, stockcode, price, chetime)

    def start(self):
        """
        이베스트 서버에 실시간 data 요청함.
        """
        self.ResFileName = "C:\\eBEST\\xingAPI\\Res\\S3_.res"  # RES 파일 등록
        self.SetFieldData("InBlock", "shcode", "005930")
        self.AdviseRealData()   # 실시간데이터 요청

    def add_item(self, stockcode):
        # 실시간데이터 요청 종목 추가
        self.SetFieldData("InBlock", "shcode", stockcode)
        self.AdviseRealData()

    def remove_item(self, stockcode):
        # stockcode 종목만 실시간데이터 요청 취소
        self.UnadviseRealDataWithKey(stockcode)

    def end(self):
        self.UnadviseRealData()  # 실시간데이터 요청 모두 취소

    @classmethod
    def get_instance(cls):
        xreal = win32com.client.DispatchWithEvents("XA_DataSet.XAReal", cls)
        return xreal



if __name__ == "__main__":
    def get_single_data():
        xq_t1101 = XQuery_t1101.get_instance()
        xq_t1101.request()

        while xq_t1101.is_data_received == False:
            pythoncom.PumpWaitingMessages()


    def get_real_data():
        xreal = XReal_S3_.get_instance()
        xreal.start()

        while True:
            pythoncom.PumpWaitingMessages()
            xreal.add_item("005930")  # 삼성전자 주식

    get_single_data()
    get_real_data()
    

종목;삼성전자,현재가;70800, 누적거래량;4003311
TR code => t1101
1 005930 70800 102540
2 005930 70800 102540
3 005930 70800 102540
4 005930 70900 102541
5 005930 70800 102542
6 005930 70900 102542
7 005930 70900 102543
8 005930 70900 102544
9 005930 70900 102545
10 005930 70900 102545
11 005930 70800 102545
12 005930 70900 102546
13 005930 70900 102546
14 005930 70900 102551
15 005930 70900 102551
16 005930 70900 102552
17 005930 70900 102552
18 005930 70800 102553
19 005930 70900 102553
20 005930 70800 102553
21 005930 70900 102553
22 005930 70900 102555
23 005930 70800 102555
24 005930 70800 102556
25 005930 70800 102557
26 005930 70900 102557
27 005930 70900 102557
28 005930 70900 102558
29 005930 70900 102559
30 005930 70900 102600
31 005930 70900 102600
32 005930 70900 102601
33 005930 70800 102603
34 005930 70800 102604
35 005930 70900 102604
36 005930 70900 102608
37 005930 70900 102609
38 005930 70900 102611
39 005930 70900 102611
40 005930 70900 102611
41 005930 70900 102611
42 005930 70800 

KeyboardInterrupt: 